In [1]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(4,4)
qc.x(0)
qc.cx(0,1)
qc.x(2)
qc.cx(2,3)
qc.x(0)
qc.z(0)
qc.cx(0,1)
qc.x(1)
qc.cx(1,2)
qc.z(1)
qc.cx(0,1)
qc.z(3)
qc.cx(2,3)

In [2]:
from primitives.circuit import QiskitCircuit
qic = QiskitCircuit(qc)

In [3]:
from percircuit import PERCircuit

In [4]:
perc = PERCircuit(qic)

In [5]:
circ = qc.copy_empty_like()
for l in perc._layers:
    circ = circ.compose(l.single_layer.qc)
    circ = circ.compose(l.cliff_layer.qc)
    circ.barrier()

circ.draw()

┌───┐      ░ ┌───┐┌───┐      ░            ░            ░ 
q_0: ┤ X ├──■───░─┤ X ├┤ Z ├──■───░────────────░────────■───░─
     └───┘┌─┴─┐ ░ └───┘└───┘┌─┴─┐ ░ ┌───┐      ░ ┌───┐┌─┴─┐ ░ 
q_1: ─────┤ X ├─░───────────┤ X ├─░─┤ X ├──■───░─┤ Z ├┤ X ├─░─
     ┌───┐└───┘ ░           └───┘ ░ └───┘┌─┴─┐ ░ └───┘└───┘ ░ 
q_2: ┤ X ├──■───░────────■────────░──────┤ X ├─░────────────░─
     └───┘┌─┴─┐ ░ ┌───┐┌─┴─┐      ░      └───┘ ░            ░ 
q_3: ─────┤ X ├─░─┤ Z ├┤ X ├──────░────────────░────────────░─
          └───┘ ░ └───┘└───┘      ░            ░            ░ 
c: 4/═════════════════════════════════════════════════════════

In [6]:
rep1,rep2,rep3 = perc.reps

In [7]:
rep1.cliff_layer.qc.draw()

q_0: ─────
          
q_1: ──■──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘
q_3: ─────
          
c: 4/═════

In [8]:
rep2.cliff_layer.qc.draw()

q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
q_2: ─────
          
q_3: ─────
          
c: 4/═════

In [9]:
rep3.cliff_layer.qc.draw()

q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
q_2: ──■──
     ┌─┴─┐
q_3: ┤ X ├
     └───┘
c: 4/═════

In [10]:
from qiskit.providers.fake_provider import FakeVigoV2
backend = FakeVigoV2()

In [11]:
from primitives.processor import QiskitProcessor
pc = QiskitProcessor(backend)

In [12]:
from processorspec import ProcessorSpec
pspec = ProcessorSpec([0,1,2,3], pc)

In [13]:
[p.to_label() for p in pspec.meas_bases]

['XXXX', 'YYXY', 'ZZXZ', 'XXYX', 'YYYY', 'ZZYZ', 'XXZX', 'YYZY', 'ZZZZ']

In [14]:
from benchmarkinstance import BenchmarkInstance
bc = BenchmarkInstance(pspec.meas_bases[0], pspec.meas_bases[0], 4, pspec, perc._layers[0].cliff_layer)

In [15]:
[p for p in pspec.meas_bases[0]]

In [16]:
bc.circuit.qc.draw()

┌───┐┌───┐      ░ ┌───┐      ░ ┌───┐      ░ ┌───┐      ░ ┌───┐┌───┐┌───┐»
q_0: ┤ H ├┤ Y ├──■───░─┤ Y ├──■───░─┤ X ├──■───░─┤ I ├──■───░─┤ Y ├┤ H ├┤ I ├»
     ├───┤├───┤┌─┴─┐ ░ ├───┤┌─┴─┐ ░ ├───┤┌─┴─┐ ░ ├───┤┌─┴─┐ ░ ├───┤├───┤├───┤»
q_1: ┤ H ├┤ X ├┤ X ├─░─┤ Y ├┤ X ├─░─┤ I ├┤ X ├─░─┤ X ├┤ X ├─░─┤ Z ├┤ H ├┤ X ├»
     ├───┤├───┤└───┘ ░ ├───┤└───┘ ░ ├───┤└───┘ ░ ├───┤└───┘ ░ ├───┤├───┤├───┤»
q_2: ┤ H ├┤ X ├──■───░─┤ I ├──■───░─┤ I ├──■───░─┤ X ├──■───░─┤ I ├┤ H ├┤ X ├»
     ├───┤├───┤┌─┴─┐ ░ ├───┤┌─┴─┐ ░ ├───┤┌─┴─┐ ░ ├───┤┌─┴─┐ ░ ├───┤├───┤├───┤»
q_3: ┤ H ├┤ X ├┤ X ├─░─┤ I ├┤ X ├─░─┤ I ├┤ X ├─░─┤ I ├┤ X ├─░─┤ I ├┤ H ├┤ I ├»
     └───┘└───┘└───┘ ░ └───┘└───┘ ░ └───┘└───┘ ░ └───┘└───┘ ░ └───┘└───┘└───┘»
c: 4/════════════════════════════════════════════════════════════════════════»
                                                                             »
«     ┌─┐         
«q_0: ┤M├─────────
«     └╥┘┌─┐      
«q_1: ─╫─┤M├──────
«      ║ └╥┘┌─┐   
«q_2: ─╫──╫─┤M├───
«      ║  ║ └╥┘┌─┐
«q_3: ─╫──╫──╫─┤M├
«      ║  ║  ║ └╥┘
«c: 4/═╩══╩══╩══╩═
«      0  1  2  3